# Making `nakeds` based on YAML settings
- from `df_opts.pkl` without `price` and `margin`

In [ ]:
import pathlib

import pandas as pd
import numpy as np

from ib_insync import util, IB, MarketOrder

from engine import Vars, get_unds, qpAsync
from support import calcsdmult_df, get_prob, get_prec
from dfrq import get_dfrq

In [ ]:
util.startLoop()

In [ ]:
# INPUTS
MARKET = 'SNP'
SYMBOL = '' # If a SYMBOL is given, this becomes a nakeds' deep-dive
RUN_ON_PAPER = False
EARLIEST = False # Used for latest expiry nakeds
RECALC_UNDS = True # If underlying prices need to be recalculated

In [ ]:
# * SETTINGS
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
locals().update(ibp.__dict__)

# set and empty log file
logf = pathlib.Path.cwd().joinpath('data', 'log', 'temp.log')
util.logToFile(path=logf, level=30)
with open(logf, "w"):
    pass

datapath = pathlib.Path.cwd().joinpath('data', MARKET.lower())

In [ ]:
df_opts = pd.read_pickle(datapath.joinpath("df_opts.pkl"))
df_unds = pd.read_pickle(datapath.joinpath("df_unds.pkl"))

In [ ]:
## ** PREPARE RAW DATA

# get dfrq
dfrq = get_dfrq(MARKET, RUN_ON_PAPER=False)

In [ ]:
# collect the symbols without blacklist

if SYMBOL: # If a symbol is given it becomes a deep-dive
    NAKEDS = set([SYMBOL])
    EARLIEST = True
    DEEPDIVE = True
else:
    NAKEDS = set(dfrq[dfrq.status == "naked"].symbol) - set(ibp.BLACKLIST)
    DEEPDIVE = False

df_raw = df_opts[df_opts.symbol.isin(NAKEDS)]

# handle EARLIEST

# . filter on dte
if EARLIEST:  # keep only the earliest expiring dte for each symbol
    df_raw = df_raw[df_raw.dte == df_raw.groupby("symbol").dte.transform(min)]

else:  # remove dtes between MINDTE and MAXDTE
    df_raw = df_raw[df_raw.dte.between(ibp.MINDTE, ibp.MAXDTE, inclusive=True)]

# handle RECALC_UNDS
und_cts = df_unds[df_unds.symbol.isin(df_raw.symbol.unique())].contract

if RECALC_UNDS:
    df_unds = get_unds(MARKET, und_cts, RUN_ON_PAPER=RUN_ON_PAPER, SAVE=False)

# update und_iv and undPrice from df_unds
df_raw.set_index('symbol', inplace=True)
df_raw.update(df_unds[['symbol', 'iv', 'undPrice']].rename(columns={'iv': 'und_iv'}).set_index('symbol'))
df_raw.reset_index(inplace=True)

# determine standard deviations
df_raw = df_raw.assign(und_sd=calcsdmult_df(df_raw.strike, df_raw.rename(columns={'und_iv':'iv'})))

# ** CLIP FOR TARGETS

remq = dfrq.set_index('symbol').remq.to_dict()

if not DEEPDIVE:
    
    # . remove calls and puts above sdMult and against direction
    call_mask = (
        (df_raw.right == "C")
        & (df_raw.und_sd > ibp.CALLSTDMULT)
        & (df_raw.strike > df_raw.undPrice)
    )
    put_mask = (
        (df_raw.right == "P")
        & (df_raw.und_sd > ibp.PUTSTDMULT)
        & (df_raw.strike < df_raw.undPrice)
    )
    
    df_raw = df_raw[call_mask | put_mask].reset_index(drop=True) 

    # integrate with remq
    df_raw = df_raw.set_index('symbol').join(dfrq.set_index('symbol').remq).reset_index()

    df_raw.loc[df_raw.right=='P','strike']*= -1

    s=(df_raw.sort_values('strike').groupby(['symbol','right'])
         .cumcount()
         .reindex(df_raw.index)
      )

    df_nakeds = df_raw[s<df_raw['symbol'].map(remq)].sort_values(['symbol','right']).reset_index(drop=True)
    df_nakeds.loc[df_nakeds.right=='P', 'strike'] *= -1

else:
    df_nakeds = df_raw.assign(remq = df_raw.symbol.map(remq))

In [ ]:
# ** GET PRICE, IV AND MARGIN

# price and iv
with IB().connect(HOST, PORT, CID) as ib:
    df_pr = ib.run(qpAsync(ib, df_nakeds.contract, **{'FILL_DELAY': 5.5}))
    ib.disconnect()

In [ ]:
# margins
orders = [MarketOrder("SELL", lot / lot)
            if MARKET.upper() == "SNP"
                else MarketOrder("SELL", lot)
            for lot in df_nakeds.lot]

opt_cos = [(c, o) for c, o in zip(df_nakeds.contract, orders)]

In [ ]:
from engine import margin, post_df, executeAsync

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    df_mgn = ib.run(
        executeAsync(
            ib=ib,
            algo=margin,
            cts=opt_cos,
            CONCURRENT=200,
            TIMEOUT=6.5,
            post_process=post_df,
            DATAPATH=datapath,
            OP_FILENAME="",
            SHOW_TQDM=True,
            **{"FILL_DELAY": 6.48},
        )
    )

In [ ]:
df_mgn.head()

In [ ]:
df_pr.head()

In [ ]:
# integrate price, iv and margins
df_nakeds = df_nakeds.set_index("conId")\
                .join(df_mgn.set_index("conId")[["comm", "margin"]])\
                     .join(df_pr.set_index("conId")\
                           [["bid", "ask", "close", "last", "iv", "price"]])\
                            .reset_index()

In [ ]:
df_nakeds

In [ ]:
# update null iv with und_iv
m_iv = df_nakeds.iv.isnull()
df_nakeds.loc[m_iv, "iv"] = df_nakeds[m_iv].und_iv

## ** GET EXPECTED PRICE AND ROM

# compute sdMult
df_nakeds = df_nakeds.assign(sdMult=calcsdmult_df(df_nakeds.strike, df_nakeds))

# compute prop
df_nakeds = df_nakeds.assign(prop=df_nakeds.sdMult.apply(get_prob))

# compute intrinsic values
df_nakeds = df_nakeds.assign(
    intrinsic=np.where(
        df_nakeds.right == "C",
        (df_nakeds.undPrice - df_nakeds.strike).clip(0, None),
        (df_nakeds.strike - df_nakeds.undPrice).clip(0, None),
    )
)

# compute time values
df_nakeds = df_nakeds.assign(timevalue=df_nakeds.price - df_nakeds.intrinsic)

In [ ]:
# compute rom based on timevalue, remove zero rom and down-sort on it
df_nakeds["rom"] = (
    (df_nakeds.timevalue * df_nakeds.lot - df_nakeds.comm).clip(0)
    / df_nakeds.margin
    * 365
    / df_nakeds.dte
)

df_nakeds = (
    df_nakeds[df_nakeds.rom > 0]
    .sort_values("rom", ascending=False)
    .reset_index(drop=True)
)

# establish expRom
#    ... for those whose RoM is < MINEXPROM, make it equal to MINEXPROM
df_nakeds["expRom"] = np.maximum(ibp.MINEXPROM, df_nakeds.rom)

# set expPrice to be based on expRom
df_nakeds["expPrice"] = (
    df_nakeds.expRom
    * np.maximum(ibp.MINOPTSELLPRICE, df_nakeds.price)
    / df_nakeds.rom
).apply(lambda x: get_prec(x, ibp.PREC))

In [ ]:
df_nakeds[df_nakeds.expRom == df_nakeds.rom]